In [11]:
import os
os.chdir(r"C:\Users\pauli\Documents\M2\fouille de données\projet\fichier")


In [ ]:
import csv
import gzip
import pickle
import random
import sys
import numpy as np
import warnings
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import Normalizer
from sklearn.metrics import average_precision_score, confusion_matrix, mean_squared_error, recall_score, f1_score, make_scorer,precision_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
from functions import loadCsv, oneHotEncodeColumns, data_recovery



In [ ]:

df = pd.read_csv('df_clean.csv', sep=",")
index_train = pd.read_csv('index_train.csv',sep=",")
index_test = pd.read_csv('index_test.csv',sep=",")

ytrain = df.loc[index_train, "FlagImpaye"]
ytest = df.loc[index_test, "FlagImpaye"]
Xtrain = df.loc[index_train, :]
# Drop the 'FlagImpaye' column
Xtrain = Xtrain.drop('FlagImpaye', axis=1)
Xtest = df.loc[index_test, :]
# Drop the 'FlagImpaye' column
Xtest = Xtest.drop('FlagImpaye', axis=1)


#data500000=data.sample(n=500000)
#data500000.reset_index(drop=True, inplace=True)


In [ ]:


from imblearn.over_sampling import SMOTE
sm = SMOTE(sampling_strategy=0.5, k_neighbors=5, random_state=1)
X = X.reset_index(drop=True)
ytrain = ytrain.reset_index(drop=True)
smote_Xtrain, smote_ytrain = sm.fit_resample(X, ytrain)
smote = pd.concat([smote_Xtrain, smote_ytrain], axis = 1)


In [12]:
warnings.filterwarnings("ignore")

seed = 1

# On stocke ci-dessous les valeurs des hyper-paramètres que l'on souhaitent tester
#np.arange(start = 5, stop = 250, step = 50)
params_modeles = [
{'estimator__max_features' : [5,10,20],
'estimator__min_samples_split' : [50],
'estimator__max_depth':[1000,30],
'estimator__min_samples_leaf':[10]},
{"estimator__solver":["newton-cg"],
 "estimator__penalty":["none"],
 "estimator__max_iter":[10000]},
{"estimator__kernel":["linear"]}
]

modeles_list = [
    DecisionTreeClassifier(),
    LogisticRegression(),
    SVC()
]

# On va stocker les résultats obtenus pour chaque jeux de données et pour chaque algorithme
results = {}

np.random.seed(seed)
random.seed(seed)

#train test split
Xtrain = pd.read_csv("smote_Xtrain.csv", sep=",")
ytrain = pd.read_csv("smote_ytrain.csv", sep=",")
Xtest = pd.read_csv("Xtest.csv", sep=",")
ytest = pd.read_csv("ytest.csv", sep=",")
Xtrain.info()
train = pd.concat([Xtrain, ytrain], axis=1)



def select_model(modeles, parameters, Xtrain, ytrain, Xtest, ytest) :
    df = pd.DataFrame(columns = ['best','score', 'ft_imp', 'ftest', 'ftrain'])
    
    #normaliser
    normalizer = Normalizer() 
    normalizer.fit(Xtrain)    
    Xtrain = normalizer.transform(Xtrain)
    Xtest = normalizer.transform(Xtest)
    
    
    for i, modele in enumerate(modeles):
        f1 = make_scorer(f1_score , average='macro')
        #select only usefull features
        sel = RFECV(estimator=modele, step=1, cv=5,scoring = f1)
        #test all models with CV
        model = GridSearchCV(estimator=sel,
                            param_grid=parameters[i],
                            scoring = f1,
                            verbose = False,
                            cv = 5)
        model.fit(XTrain, yTrain)
        
        rankTrain = model.predict(Xtrain)
        rankTest = model.predict(Xtest)

        #calcul de la f-mesure pour mesurer la performance du modele 
        ctrain = confusion_matrix(ytrain, rankTrain)
        ftrain = round(2*ctrain[1,1]/(2*ctrain[1,1]+ctrain[0,1]+ctrain[1,0]),4)
        ctest = confusion_matrix(ytest, rankTest)
        ftest = round(2*ctest[1,1]/(2*ctest[1,1]+ctest[0,1]+ctest[1,0]),4)

        #calcul indicators 
        pred = model.predict(XTest)
        features=list(X.columns[model.best_estimator_.support_])
        #df with all indicators
        df.loc[i]=[model.best_estimator_, model.best_score_, features, ftest, ftrain]
    #return df with all indicators  
    return df

df_ind = select_model(modeles_list, params_modeles, X_smote, y_smote)


#appliquer et sauvegarder le meilleur modele
#get index of df wich have best f1 score
indice=df_ind['ftest'].idxmax()
#keep model with best f score
best_model = df_ind["best"][indice]
best_model.fit(X, y)
pred = best_model.predict(X_sub)
pickle.dump(best_model, open("./model.pickle.dat", "wb"))


In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Sun Dec 25 09:25:17 2022

@author: pauli
"""

import pandas as pd
Xtrain = pd.read_csv("smote_Xtrain.csv", sep=",")
ytrain = pd.read_csv("smote_ytrain.csv", sep=",")
Xtrain.info()
train = pd.concat([Xtrain, ytrain], axis=1)


#XGBoost/ gradient tree boosting 
from sklearn.ensemble import GradientBoostingClassifier 
param = {"loss":"log_loss","learning_rate":0.1,"n_estimators":100,"min_samples_split":2}
gbc = GradientBoostingClassifier(loss="log_loss", learning_rate=0.1, n_estimators=100, min_samples_split=2)
gbc.fit(Xtrain, ytrain)

#Nearest-Neighbor
from sklearn.neighbors import KNeighborsClassifier
knc = KNeighborsClassifier(n_neighbors=2, algorithm='ball_tree')
knc.fit(Xtrain, ytrain)

#Decision Trees
from sklearn.tree import DecisionTreeClassifier
param = {"criterion":"gini","max_depth":None,"min_samples_split":2,"min_samples_leaf":1,"max_features":"sqrt"}
dtc = DecisionTreeClassifier()
dtc = dtc.fit(Xtrain, ytrain)

#Random Forests
from sklearn.ensemble import RandomForestClassifier
param = {"n_estimators":100,"criterion":"gini","max_depth":None,"min_samples_split":2,"min_samples_leaf":1,"max_features":"sqrt","oob_score":False,"warm_start":False,"max_samples":None}
rfc = RandomForestClassifier()
rfc = rfc.fit(Xtrain, ytrain)

#SVM
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
param = {"kernel":"rbf","degree":3,}
#ici on peut changer le noyaux
svc = make_pipeline(StandardScaler(), SVC(kernel="rbf",degree=3))
svc.fit(Xtrain, ytrain)

#K-means
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=2, random_state=0, n_init="auto")
kmeans.fit(Xtrain)
#bof ca, on ne prend meme pas en compte les y...

#Sampling
###A FORCEMENT FAIRE !!!!!

#LOF
from sklearn.neighbors import LocalOutlierFactor
lof = LocalOutlierFactor(n_neighbors=2)
lof.fit(train) #train contient X et y 

#Auto-encodeurs
#keskecé ???

#Reseaux de neurones
from sklearn.neural_network import MLPClassifier
mlpc = MLPClassifier(random_state=1, max_iter=100)
mlpc.fit(Xtrain, ytrain)

#ADL
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
adl = LinearDiscriminantAnalysis()
adl.fit(Xtrain, ytrain)

#ADQ
from sklearn.qda import QDA
qda = QDA()
qda.fit(Xtrain, ytrain)

#Cost-sensitive learning
#On pondère les erreurs 
#Modifier le poids de chaque classe sur le substitue de taux d’erreur 
#Attribuer un poids a chaque entrée de la matrice de confusion (cout a l’échelle de chaque classe) 

#Methodes ensemblistes
#bagging
from sklearn.ensemble import BaggingClassifier
from sklearn.neighbors import KNeighborsClassifier
param = {"max_features":0.5,"max_samples" : 0.5}
bagging = BaggingClassifier(KNeighborsClassifier())

#boosting 
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import cross_val_score
adab = AdaBoostClassifier(n_estimators=100)
scores = cross_val_score(adab, Xtrain, ytrain, cv=5)

#regression logistique
from sklearn.linear_model import LogisticRegression
param = {"solver":"saga","penalty":"none","max_iter":100}
logit = LogisticRegression(solver="saga", penalty="none", max_iter=100, random_state=1)
logit.fit(Xtrain, ytrain)


#Metric Learning
from sklearn.metrics import mean_squared_error, recall_score, f1_score, make_scorer,precision_score
#mesure de perg : 
    #Accuracy
    #précision
    #rappel
    #F-mesure = (2TP)/(2TP + FN + FP)
    #courbe ROC (AUC ROC).

